In [1]:
with open('input.txt') as fl:
    bricks = [[[int(c) for c in brick.split(',')] for brick in ln.strip().split('~')  ] for i, ln in enumerate(fl.readlines())]
bricks = {i:brick for i, brick in enumerate(sorted(bricks, key=lambda x: min(x[0][2], x[1][2])))}
mx = max([max(b[0][0],b[1][0]) for b in bricks.values()])
my = max([max(b[0][1],b[1][1]) for b in bricks.values()])
mz = max([max(b[0][2],b[1][2]) for b in bricks.values()])

In [2]:
xylookup = {}
missed = 0
for x in range(mx+1):
    for y in range(my+1):
        xylookup[(x,y)] = {}
        for b, ((bx1,by1,bz1), (bx2,by2,bz2)) in bricks.items():
            if bx1==bx2 and by1==by2 and x==bx1 and y==by1:
                # single point brick (vertical)
                xylookup[(x,y)][b] = (min(bz1, bz2), max(bz1, bz2))
            elif bz1==bz2 and by1==by2 and min(bx1,bx2)<=x<=max(bx1,bx2) and y==by1: 
                # brick in x dim
                xylookup[(x,y)][b] = (bz1, bz1)
            elif bz1==bz2 and bx1==bx2 and min(by1,by2)<=y<=max(by1,by2) and x==bx1:
                # brick in y dim
                xylookup[(x,y)][b] = (bz1, bz1)
            else:
                missed += 1
        xylookup[(x,y)] = {p:z for p,z in sorted(xylookup[(x,y)].items(), key=lambda x: x[1][0])}

In [3]:
# go through each brick (which is defined to be in starting z order) and drop them
# if they hit another, store the z network
# don't forget to update the x,y lookup
support_network = {}
for b, ((bx1,by1,bz1), (bx2,by2,bz2)) in bricks.items():
    bzb, bzt = min(bz1, bz2), max(bz1, bz2)
    if min(bz1,bz2) == 1:
        continue
    # get bricks below
    below = {}
    for x in range(min(bx1,bx2), max(bx1,bx2)+1):
        for y in range(min(by1,by2), max(by1,by2)+1):
            for bb, (bbz1, bbz2) in xylookup[(x,y)].items():
                bbzt = max(bbz1,bbz2)
                if bzb > bbzt:
                    below[bb] = bbzt
    # drop the brick to the bottom
    if len(below) > 0:
        mzb = max(below.values())
        below = [b for b, z in below.items() if z==mzb]
        # now drop the brick and update the support network and lookups
        nzb,nzt = mzb+1,mzb+1+bzt-bzb
        support_network[b] = below
    else:
        nzb,nzt = 1,1+bzt-bzb
    bricks[b] = [[bx1,by1,nzb],[bx2,by2,nzt]]
        
    # update the lookup table with new heights
    for x in range(min(bx1,bx2), max(bx1,bx2)+1):
        for y in range(min(by1,by2), max(by1,by2)+1):
            if b in xylookup[(x,y)]:
                xylookup[(x,y)][b] = (nzb, nzt)
    

In [4]:
# part 1
critical = set()
for s in support_network.values():
    if len(s) == 1:
        critical.add(s[0])
len(set(bricks.keys()) -  critical)

418

In [5]:
# part 2
def chain(start):
    p_len = 0
    gone = set([start])
    while len(gone) != p_len:
        p_len = len(gone)
        for b, s in support_network.items():
            if len(set(s) - gone) == 0:
                gone.add(b)
    gone.remove(start)
    return gone
sum([len(chain(s)) for s in bricks])

70702